In [1]:
using Test

In [2]:
f = readlines("day11.input");

In [3]:
mutable struct Item
    worry::Int64
end


In [4]:
mutable struct Monkey
    times_inspected::Int64
    items::Vector{Item}
    operation::Function
    test_dividend::Int64
    true_monkey::Int64
    false_monkey::Int64
    Monkey(operation, test_dividend, true_monkey, false_monkey) = new(0, Vector{Item}(), operation, test_dividend, true_monkey, false_monkey)
end

In [5]:
function parse_input(input)
    monkeys = Dict{Int64, Monkey}()
    monkey_index = 0
    i = 1
    while i < length(input)
        i += 1
        line = last(split(input[i], ":"))
        items = [parse(Int64, num) for num in split(line, ",")]

        i += 1
        contains(input[i], "Operation") || error("bad")
        operation_line = last(split(input[i], "="))
        operand1, operator, operand2 = split(operation_line)
        if operator == "*"
            parsed_operator = *
        else
            parsed_operator = +
        end
        if operand1 == "old" && operand2 == "old"
            monkey_operation = x -> parsed_operator(x, x)
        elseif operand1 == "old"
            monkey_operation = x -> parsed_operator(x, parse(Int64, operand2))
        elseif operand2 == "old"
            monkey_operation = x -> parsed_operator(x, parse(Int64, operand1))
        else
            monkey_operation = x -> parsed_operator(parse(Int64, operand1), parse(Int64, operand2))
        end

        i += 1
        line = last(split(input[i]))
        dividend = parse(Int64, line)

        i += 1
        line = last(split(input[i]))
        true_monkey = parse(Int64, line)

        i += 1
        line = last(split(input[i]))
        false_monkey = parse(Int64, line)

        monkeys[monkey_index] = Monkey(monkey_operation, dividend, true_monkey, false_monkey)
        for item in items
            push!(monkeys[monkey_index].items, Item(item))
        end
        monkey_index += 1
        i += 2
    end
    monkeys
end


parse_input (generic function with 1 method)

In [6]:
test_monkey = parse_input(String.(split("Monkey 0:
Starting items: 83, 88, 96, 79, 86, 88, 70
Operation: new = old * 5
Test: divisible by 11
  If true: throw to monkey 2
  If false: throw to monkey 3", "\n")))[0]

test_monkey.operation(5)

25

In [7]:
function solve(input, rounds, worry_adjustment)
    monkeys = parse_input(input)
    mega_number = map(*, (monkey.test_dividend for monkey in values(monkeys))...)
    for round in 1:rounds
        for monkey_index in 0:length(monkeys)-1
            monkey = monkeys[monkey_index]
            for _= 1:length(monkey.items)
                monkey.times_inspected += 1
                item = popfirst!(monkey.items)
                item.worry %= mega_number
                item.worry = monkey.operation(item.worry)
                item.worry = worry_adjustment(item.worry)

                if item.worry % monkey.test_dividend == 0
                    target_monkey = monkey.true_monkey
                else
                    target_monkey = monkey.false_monkey
                end
                
                push!(monkeys[target_monkey].items, item)
            end

            length(monkey.items) == 0 || error("items remaining")
        end

        # if round == 20 || round%1000==0
        #     for idx=0:length(monkeys)-1
        #         value = monkeys[idx]
        #         println("After $round rounds, monkey $idx inspected $(value.times_inspected) times.")
        #     end
        # end
    end

    best_monkeys = first(sort(collect(values(monkeys)), by = m -> -m.times_inspected), 2)
    best_monkeys[1].times_inspected * best_monkeys[2].times_inspected
end

solve (generic function with 1 method)

In [8]:
function solve_part_1(input)
    solve(input, 20, worry -> worry ÷ 3)
end

solve_part_1 (generic function with 1 method)

In [9]:
function solve_part_2(input)
    solve(input, 10000, worry -> worry)
end

solve_part_2 (generic function with 1 method)

In [10]:
@test solve_part_1(String.(split("Monkey 0:
Starting items: 79, 98
Operation: new = old * 19
Test: divisible by 23
  If true: throw to monkey 2
  If false: throw to monkey 3

Monkey 1:
Starting items: 54, 65, 75, 74
Operation: new = old + 6
Test: divisible by 19
  If true: throw to monkey 2
  If false: throw to monkey 0

Monkey 2:
Starting items: 79, 60, 97
Operation: new = old * old
Test: divisible by 13
  If true: throw to monkey 1
  If false: throw to monkey 3

Monkey 3:
Starting items: 74
Operation: new = old + 3
Test: divisible by 17
  If true: throw to monkey 0
  If false: throw to monkey 1", "\n"))) == 10605


96577

Test Passed

In [11]:
(solve_part_1(f), solve_part_2(f))

96996909699690

(64032, 12729522272)

In [12]:
@test solve_part_2(String.(split("Monkey 0:
Starting items: 79, 98
Operation: new = old * 19
Test: divisible by 23
  If true: throw to monkey 2
  If false: throw to monkey 3

Monkey 1:
Starting items: 54, 65, 75, 74
Operation: new = old + 6
Test: divisible by 19
  If true: throw to monkey 2
  If false: throw to monkey 0

Monkey 2:
Starting items: 79, 60, 97
Operation: new = old * old
Test: divisible by 13
  If true: throw to monkey 1
  If false: throw to monkey 3

Monkey 3:
Starting items: 74
Operation: new = old + 3
Test: divisible by 17
  If true: throw to monkey 0
  If false: throw to monkey 1", "\n"))) == 2713310158

96577

Test Passed